PREGUNTAR: Puedo chorearme mi archivo punto2/contabilizado/output.txt con estructura

```sql 
id_retador puntos_total cantidad
```

para simplificar el calculo de la PP?

Primer JOB es solo para formatear el DATASET COMO UN INNER JOIN 

In [71]:
from MRE import Job
inputDir = "./jugadores/"
promedioDir = "../punto2/contabilizado/"
outputDir = "./format/"

In [72]:
def fmap(key, value, context):
    lista = value.split()
    #print(key)
    context.write((int(key),'RETADO'), lista[0]) 

def fmap2(key, value, context):
    lista = value.split()
    #print("2 " + key)

    context.write((int(key), 'PP')  , (int(lista[0])/int(lista[1])))

def cmpShort(key1,key2):
    if(key1[0] == key2[0]):
        return 0
    elif(key1[0] < key2[0]):
        return -1
    else:
        return 1

def cmpShuffle(key1, key2):
    if(key1[1] == key2[1]):
        return 0
    elif(key1[1] == "PP"): #PP es el primero 
        return -1
    else:
        return 1 #Caso retado

def fred(key, values, context):
    acumular_retados = set()
    pp = next(values)  #Es un iterator pero si lo resuelvo con next() me da cualquier cosa 
    for v in values:
        acumular_retados.add(v)
    print(f"KEY: {key} PP: {pp} RETADOS: {acumular_retados}")
    context.write(key[0], (pp,acumular_retados,puntaje_heroico_inicial))

    
puntaje_heroico_inicial = 1

jobCount = Job(inputDir, outputDir, fmap, fred)
jobCount.setSortCmp(cmpShuffle)
jobCount.setShuffleCmp(cmpShort)
jobCount.setParams(puntaje_heroico_inicial)
jobCount.addInputPath(promedioDir,fmap2)

success = jobCount.waitForCompletion()

print(success)

KEY: (1, 'RETADO') PP: 1919.2222222222222 RETADOS: {'15', '48', '49', '23', '41', '42', '47', '37', '73', '43', '96', '57', '2', '10', '64'}
KEY: (3, 'RETADO') PP: 2524.5 RETADOS: {'46', '62', '21', '28'}
KEY: (4, 'RETADO') PP: 2663.0 RETADOS: {'5', '56'}
KEY: (5, 'PP') PP: 3288.0 RETADOS: {'62'}
KEY: (7, 'RETADO') PP: 1231.5 RETADOS: {'44', '69'}
KEY: (8, 'RETADO') PP: 2060.25 RETADOS: {'57', '36', '76', '38'}
KEY: (9, 'RETADO') PP: 2468.0 RETADOS: {'62', '40', '42', '47', '58', '74'}
KEY: (10, 'RETADO') PP: 1325.6666666666667 RETADOS: {'64', '60', '8'}
KEY: (11, 'PP') PP: 1411.0 RETADOS: {'49'}
KEY: (12, 'RETADO') PP: 2886.3333333333335 RETADOS: {'25', '48', '50'}
KEY: (13, 'RETADO') PP: 3858.6666666666665 RETADOS: {'1', '49', '55'}
KEY: (14, 'RETADO') PP: 2518.3333333333335 RETADOS: {'45', '55', '37'}
KEY: (15, 'RETADO') PP: 992.5 RETADOS: {'62', '89'}
KEY: (16, 'PP') PP: 3567.5 RETADOS: {'76', '89'}
KEY: (17, 'RETADO') PP: 2431.6666666666665 RETADOS: {'26', '23', '33', '27', '39', 

Segundo job iterativo

In [73]:
inputDir = "./format/"
outputDir = "./out/"

In [74]:
def fmap(key, value, context):
    context.write(key, value)

def fred(key, values, context):
    c = 0
    for v in values:
        c += 1
    if c >= H:
        context.write(key, c)
H = 12

alfa = 0.1
jobCountBoth = Job(inputDir, outputDir, fmap, fred)
jobCount.setParams(alfa)
jobCountBoth.setParams(H)
success = jobCountBoth.waitForCompletion()

print(success)

True


Tercer Job saca top 10 